## Accuracy: 

things to change:
1. Kilometers/Charge Delta is blank for original script
2. str leg status is not matched
3. LastMod_SalesUser is not matched
4. TBL_RES_SEGMENTS join issue

things to note:
1. Route Pair doesn't matter?

In [3]:
from helpfile import *

# Import and clean snowflake views

In [ ]:
query = """select * from ANALYTICS_PROD.IOATAWARE.VW_RESERVATION_DETAIL_RESERVATION_NUMBER where "Reservation Nmbr"=6297651
    """
snowflake_pd_df = get_data(query)

In [ ]:
columns_to_remove = ['Charge Delta','Kilometers','str Leg Status','Last Mod Sales User','Route Pair']
snowflake_pd_df = snowflake_pd_df.drop(columns=columns_to_remove)

snowflake_pd_df['Charge Date'] = pd.to_datetime(snowflake_pd_df['Charge Date'])

snowflake_pd_df['Flight Date'] = pd.to_datetime(snowflake_pd_df['Flight Date'])
snowflake_pd_df['Flight Date'] =snowflake_pd_df['Flight Date'].dt.strftime('%m/%d/%Y')
snowflake_pd_df['Flight Date'] = pd.to_datetime(snowflake_pd_df['Flight Date'])

snowflake_pd_df['Net Charge'] = snowflake_pd_df['Net Charge'].astype(float)
snowflake_pd_df['Taxes'] = snowflake_pd_df['Taxes'].astype(float)
snowflake_pd_df['Total Charge'] = snowflake_pd_df['Total Charge'].astype(float)
snowflake_pd_df['Flight_Nmbr'] = snowflake_pd_df['Flight_Nmbr'].astype(int)
snowflake_pd_df['Base NetCharge'] = snowflake_pd_df['Base NetCharge'].astype(float)
snowflake_pd_df['Ancillary NetCharge'] = snowflake_pd_df['Ancillary NetCharge'].astype(float)
# snowflake_pd_df['Route Pair'] = snowflake_pd_df['Route Pair'].str.replace('  ', '')
# snowflake_pd_df['Route Pair'] = snowflake_pd_df['Route Pair'].str.replace(' ', '')
snowflake_pd_df['Description'] = snowflake_pd_df['Description'].str.replace(r'^\s+', '', regex=True)


snowflake_pd_df['Percent'] = snowflake_pd_df['Percent'].replace('%', '').astype(float)
snowflake_pd_df['Exchange Rate'] = snowflake_pd_df['Exchange Rate'].replace('%', '').astype(float)
snowflake_pd_df['Tax Percentage'] = snowflake_pd_df['Tax Percentage'].str.replace('%', '').astype(float)

snowflake_pd_df['Base Charge'] = snowflake_pd_df['Base Charge'].astype(float)
snowflake_pd_df['Base Discount'] = snowflake_pd_df['Base Discount'].astype(float)
snowflake_pd_df['Base Taxes'] = snowflake_pd_df['Base Taxes'].astype(float)

In [ ]:
#snowflake_pd_df.dtypes

# Import and clean report data

In [ ]:
report_df = pd.read_csv('Reservation_Detail/Reservation_Detail_Reservation_Number_6297651.csv', skiprows=0)   

In [ ]:
columns_to_remove = ['str_Promo_Code','CASM','ChargeDelta','Kilometers','str_Leg_Status','LastMod_SalesUser','RoutePair']
report_df = report_df.drop(columns=columns_to_remove)

columns_to_process = ['NetCharge', 'Taxes','TotalCharge1','Base_NetCharge','Ancillary_NetCharge','Base_Charge','Base_Discount','Base_Taxes']

# Apply the function to specified columns in the dataframe
for col in columns_to_process:
    report_df[col] = report_df[col].apply(process_fee_column)

report_df['ChargeDate'] = pd.to_datetime(report_df['ChargeDate'])
report_df['ChargeDate'] =report_df['ChargeDate'].dt.strftime('%m/%d/%Y')
report_df['ChargeDate'] = pd.to_datetime(report_df['ChargeDate'])
report_df['FlightDate'] = pd.to_datetime(report_df['FlightDate'])
report_df['FlightDate'] =report_df['FlightDate'].dt.strftime('%m/%d/%Y')
report_df['FlightDate'] = pd.to_datetime(report_df['FlightDate'])
# report_df['RoutePair'] = report_df['RoutePair'].str.replace(' ', '')

report_df['Percent'] = report_df['Percent'].str.replace('%', '').astype(float)
report_df['mny_Exchange_Rate'] = report_df['mny_Exchange_Rate'].str.replace('%', '').astype(float)
report_df['mny_Tax_1_Percentage'] = report_df['mny_Tax_1_Percentage'].str.replace('%', '').astype(float)

report_df['mny_Distance'] = report_df['mny_Distance'].str.replace(',', '').astype(int)
report_df['Exchange_Rate'] = report_df['Exchange_Rate'].str.replace('%', '').astype(float)
report_df['Leg_TotalDistance'] = report_df['Leg_TotalDistance'].str.replace(',', '').astype(int)

In [ ]:
#report_df.dtypes

# Compare two dataframes

In [8]:
report_accuracy,snowflake_accuracy,report_only,snowflake_only = calculate_accuracy(report_df,snowflake_pd_df)
print(f"Accuracy: {snowflake_accuracy:.2f}%" " snowflake records in report\nThe number of records in snowflake but not in report is", len(snowflake_only))
print(f"Accuracy: {report_accuracy:.2f}%"" report records in snowflake\nThe number of records in report but not in snowflake is", len(report_only))

Accuracy: 78.57% snowflake records in report
The number of records in snowflake but not in report is 12
Accuracy: 73.33% report records in snowflake
The number of records in report but not in snowflake is 16


In [27]:
report_df.columns

Index(['ChargeDate', 'lng_Reservation_Nmbr', 'lng_Sked_Detail_Id_Nmbr',
       'FlightDate', 'Departure', 'Arrival', 'lng_Res_Legs_Id_Nmbr',
       'ChargeType', 'NetCharge', 'Taxes', 'TotalCharge1', 'lng_Leg_Nmbr',
       'str_GL_Charge_Type_Desc', 'str_Flight_Nmbr', 'str_GL_Charges_Desc',
       'Reference', 'str_Currency_Ident', 'lng_Res_Segments_Id_Nmbr',
       'Transborder', 'str_Airline_Codes_Ident', 'mny_Distance',
       'Leg_TotalDistance', 'str_Agency_Name', 'Percent', 'Category',
       'lng_GL_Charge_Type_Id_Nmbr', 'str_Tax_Configuration_Name',
       'mny_Tax_1_Percentage', 'mny_Exchange_Rate', 'LegDeparture',
       'LegArrival', 'Description', 'Channel', 'Local', 'AncillaryCategory',
       'PurchaseCnt', 'Base_NetCharge', 'Ancillary_NetCharge',
       'TicketPurchase', 'TicketRefund', 'Country', 'Classification',
       'Exchange_Rate', 'Base_Charge', 'Base_Discount', 'Base_Taxes'],
      dtype='object')

In [28]:
snowflake_pd_df.columns

Index(['Charge Date', 'Reservation Nmbr', 'Sked Detail Id Nmbr', 'Flight Date',
       'Departure', 'Arrival', 'Res Legs Id Nmbr', 'Charge Type', 'Net Charge',
       'Taxes', 'Total Charge', 'lng Leg Nmbr', 'GL Charge Type Desc',
       'Flight_Nmbr', 'GL Charges Desc', 'PNR', 'Currency Ident',
       'Res Segments Id Nmbr', 'Transborder', 'Airline Codes Ident',
       'Distance', 'Leg Total Distance', 'Agency_Name', 'Percent', 'Category',
       'GL Charge Type Id Nmbr', 'Tax Name', 'Tax Percentage', 'Exchange Rate',
       'Leg Departure', 'Leg Arrival', 'Description', 'Channel', 'Local',
       'Ancillary Category', 'Purchase Cnt', 'Base NetCharge',
       'Ancillary NetCharge', 'Ticket Purchase', 'Ticket Refund', 'Country',
       'Classification', 'Exchange_Rate', 'Base Charge', 'Base Discount',
       'Base Taxes'],
      dtype='object')

In [2]:
# Convert numeric columns to strings before concatenation
report_df['key'] = report_df['lng_Reservation_Nmbr'].astype(str) + '/' + report_df['str_Flight_Nmbr'].astype(str) + '/' + report_df['lng_Res_Legs_Id_Nmbr'].astype(str)
snowflake_pd_df['key'] = snowflake_pd_df['Reservation Nmbr'].astype(str) + '/' + snowflake_pd_df['Flight_Nmbr'].astype(str) + '/' + snowflake_pd_df['Res Legs Id Nmbr'].astype(str)

NameError: name 'report_df' is not defined

In [1]:
report_df

NameError: name 'report_df' is not defined

In [34]:
print(len(report_df))
len(set(report_df['key']))

60


11

# Output the difference file

In [9]:
try:
    snowflake_only.to_csv("Reservation_Detail/"+"snowflake_only_record.csv", index=False) 
except:
    print("No snowflake only records")
try:
    report_only.to_csv("Reservation_Detail/"+"report_only_record.csv", index=False) 
except:
    print("No report only records")

In [10]:
snow_res = list(snowflake_pd_df['Reservation Nmbr'].unique())
report_res = list(report_df['Reservation Nmbr'].unique())

In [11]:
elements_in_snow_not_in_report = list(set(snow_res) - set(report_res))
elements_in_report_not_in_snow = list(set(report_res)-set(snow_res))

snow_file_path = "Sales/snowonly_reservation.txt"
report_file_path = "Sales/reportonly_reservation.txt"

# Open the file for writing
with open(snow_file_path, "w") as file:
    for item in elements_in_snow_not_in_report:
        file.write("%s," % item)
with open(report_file_path, "w") as file:
    for item in elements_in_snow_not_in_report:
        file.write("%s," % item)